In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('/home/yoxara/smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V

/tmp/ipykernel_658778/906619999.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp


3.0.0-beta


In [2]:
BSMlist = getParticlesFromSLHA('/home/yoxara/smodels/smodels-database/13TeV/ATLAS/ATLAS-EXOT-2019-03/validation/slhaFiles_Simp_Axial_Vector/run_01_tag_1.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
#model.updateParticles(inputFile=slhaFile)
print(BSMlist)

[xd, xd~, y1]


In [3]:
slhaFolder = '/home/yoxara/smodels/smodels-database/13TeV/ATLAS/ATLAS-EXOT-2019-03/validation/slhaFiles_Simp_Axial_Vector'
resultsFolder = '/home/yoxara/smodels/smodels-database/13TeV/ATLAS/ATLAS-EXOT-2019-03/validation/results_smodels'
File = '/home/yoxara/smodels/smodels-database/13TeV/ATLAS/ATLAS-EXOT-2019-03/validation/parameters_simp.ini'
!chmod -R +rwx {slhaFolder}

In [4]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadTextDatabase() in 456: Parsing text database at /home/yoxara/smodels/smodels-database/
INFO in databaseObj.createBinaryFile() in 607: /home/yoxara/smodels/smodels-database/db3.pcl created.
INFO in metaObj.printFastlimBanner() in 162: FastLim v1.1 efficiencies loaded. Please cite: arXiv:1402.0492, EPJC74 (2014) 11
INFO in modelTester.loadDatabaseResults() in 498: Including non-validated results
INFO in modelTester.testPoints() in 358: Running SModelS for 464 files with a single process. Messages will be redirected to smodels.log


In [5]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)

    data.append(dataDict)
    

In [6]:
print(len(data))

464


In [7]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 7)
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_95_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",5.237883e-01,0.000000e+00,3.176134e+00,2.652346e+00,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,run_135_tag_1_3rd.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",2.427839e+00,0.000000e+00,1.473519e+01,1.230735e+01,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,run_58_tag_1.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.972809e+01,0.000000e+00,1.207296e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,0.000000e+00,1.010015e+02,2.706370e+01,1.838680e+01,[TRV1jj],"[(y1, 2750.0)]",ATLAS-EXOT-2019-03,NaN,1.300000e+01,1.360000e+02,upperLimit,3.731991e+00,5.493152e+00,"[(y1, prompt)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,run_59_tag_1_2nd.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.312555e+01,0.000000e+00,7.959039e+01,6.646484e+01,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,run_08_tag_1_2nd.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.657409e+02,0.000000e+00,2.928085e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,0.000000e+00,2.462344e+03,6.623890e+01,7.149080e+01,[TRV1jj],"[(y1, 1800.0)]",ATLAS-EXOT-2019-03,NaN,1.300000e+01,1.360000e+02,upperLimit,3.717369e+01,3.444281e+01,"[(y1, prompt)]"
463,run_61_tag_1_3rd.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",2.440482e+02,0.000000e+00,1.504581e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,/home/yox

In [8]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 5000001])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)


In [9]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [10]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [11]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 464


In [12]:
#Save DataFrame to pickle file:
dataDF.to_pickle('/home/yoxara/smodels/smodels-database/13TeV/ATLAS/ATLAS-EXOT-2019-03/validation/data_Axial_Vector_new.pcl')
print(len(dataDF))
dataDF

464


,filename,mass.5000001,mass.5000521,width.5000001,width.5000521,extpar.1,extpar.2,extpar.3,extpar.4,xsec13TeV(fb).5000001,"BRs.y1.q,q","BRs.y1.c,c","BRs.y1.b,b","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_95_tag_1.slha,3.750000e+03,1.000000e+04,3.007046e+01,0.000000e+00,0.000000e+00,1.500000e+00,1.297300e-01,0.000000e+00,3.176134e+00,1.670176e-01,1.670175e-01,1.670160e-01,1.649138e-01,3.176134e+00,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",5.237883e-01,0.000000e+00,3.176134e+00,2.652346e+00,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,run_135_tag_1_3rd.slha,3.600000e+03,1.000000e+04,9.759062e+01,0.000000e+00,0.000000e+00,1.500000e+00,2.385490e-01,0.000000e+00,1.473519e+01,1.670474e-01,1.670473e-01,1.670457e-01,1.647647e-01,1.473519e+01,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",2.427839e+00,0.000000e+00,1.473519e+01,1.230735e+01,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,run_58_tag_1.slha,2.750000e+03,1.000000e+04,9.445584e+01,0.000000e+00,0.000000e+00,1.500000e+00,2.687360e-01,0.000000e+00,1.207296e+02,1.673192e-01,1.673190e-01,1.673162e-01,1.634073e-01,1.207296e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.972809e+01,0.000000e+00,1.207296e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,/home/yoxara/smodels/smodels-database/13TeV/AT...,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,/home/yoxara/smodels/smodels-database/13TeV/AT...,3.0.0-beta,3.0.0-beta,0.000000e+00,1.010015e+02,2.706370e+01,1.838680e+01,[TRV1jj],"[(y1, 2750.0)]",ATLAS-EXOT-2019-03,NaN,1.300000e+01,1.360000e+02,upperLimit,3.731991e+00,5.493152e+00,"[(y1, prompt)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,run_59_tag_1_2nd.slha,3.750000e+03,1.000000e+04,7.548940e+02,0.000000e+00,0.000000e+00,1.500000e+00,6.500000e-01,0.000000e+00,7.959038e+01,1.670176e-01,1.670175e-01,1.670160e-01,1.649138e-01,7.959038e+01,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.312555e+01,0.000000e+00,7.959039e+01,6.646484e+01,0.000000e+00